# Intro

the membrane-embedded nature of membrane proteins is not accounted for in traditional normal mode analysis.

As far as we can tell, two approaches have been implemented in [ProDy](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3309413/). The first one, [imANM](http://prody.csb.pitt.edu/tutorials/membrane_anm/imanm.html), separates the spring constant into 3 components and the component normal to the membrane is rescaled to dampen radial motions. The other one, [exANM](http://prody.csb.pitt.edu/tutorials/membrane_anm/exanm.html), models the whole slab of lipid membrane by space-filling it with beads, computes the Hessian of the whole system and reduces it yielding the effective Hessian of the protein in the membrane environment.

Although the approach in exANM is appealing, it is costly and requires to model the full membrane slab beforehand. We propose here a slightly different approach.

The idea would be to wrap the transmembrane region of the protein in a mesh of pseudo-particles harmonically constrained to stay in place and tightly coupled to the neighbouring protein atoms. Here also, the total Hessian is reduced to yield the effective Hessian of the protein, but this time at a much reduced cost.

# Methods

## Building the membrane skin

The molecular surface of the transmembrane part of the protein is found using a big probe radius, (e.g. see algorithm in  [epock](https://www.ncbi.nlm.nih.gov/pubmed/25505095) [code](https://bitbucket.org/epock/epock)) and the voxels identified are kept as anchor-particles.

## Building the Hessian

The total Hessian can be seen as the block matrix $H=\left[\begin{array}{c c} H_{PP} & H_{PM} \\ H_{MP} & H_{MM} \end{array}\right]$ with Protein-Protein, Protein-Membrane, Membrane-Protein and Membrane-Membrane terms. $H_{PP}$ is built as commonly done - pairing all atoms in the Protein within a given cutoff weigthed with spring constant $k_{PP}$, but we need to explicit the other parts (To be done in greater detail). 

In a nutshell, the coupling $H_{PM}$ involves the coupling between each membrane particle and any atoms from the protein within a given cutoff, weighted with a strong spring constant $k_{PM}$.

The membrane part $H_{MM}$ follows a slightly different definition as it constraints the *particle position*, not the *interparticle distance* as in the other terms. Something like $H_{MM}(i,j) = \frac{\partial E_{M}}{\partial x_{i}\partial x_{j}}$ where $E_{M} = \sum_{i} k_{i}|\mathbf{r}_{i}-\mathbf{r}_{i}^{(0)}|^{2}$.

Then the total Hessian is reduced to the effective protein Hessian $H_{eff} = H_{PP} - H_{PM}H_{MM}^{-1}H_{MP}$.

## Calculating the modes

Once the effective Hessian is built, things become normal again, and normal modes are obtained by solving the eigevanlue problem.